# Imports

In [19]:
# !pip install --upgrade google-cloud-aiplatform
import os
import json
import time
from datetime import datetime

In [ ]:
# import os
# import json
# import time
# from datetime import datetime
# import openai
# import pandas as pd
# import numpy as np

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, ChatSession, Part
from vertexai.language_models import TextEmbeddingModel

In [4]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/spanwar/Documents/collage/projects/chatbot_v2/vertex_key/oval-smile-417517-43cf867c95fd.json'

In [5]:
# with open(os.environ['GOOGLE_APPLICATION_CREDENTIALS'], 'r') as f:
#     print(f.read())

In [6]:
# TODO(developer): Update and un-comment below lines
project_id = "oval-smile-417517"
location   = "us-central1"
vertexai.init(project=project_id, location=location)

model = GenerativeModel("gemini-pro")

In [4]:
def get_chat_response(prompt: str):
    response = model.generate_content(prompt)
    return response

# prompt = "hi"
# response = get_chat_response(prompt)

In [5]:
def response_to_text(response):
        return str(response.candidates[0].content.parts[0]).split('text: "')[-1].split('"\n')[0]

# response_to_text(response)

In [3]:
model_text_emb = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")

NameError: name 'TextEmbeddingModel' is not defined

In [10]:
def text_embedding(text) -> list:
    """Text embedding with a Large Language Model."""
    embeddings = model_text_emb.get_embeddings([text])
    for embedding in embeddings:
        vector = embedding.values
        print(f"Length of Embedding Vector: {len(vector)}")
    return vector

In [11]:
# text_embedding('hi')

In [12]:


messages = [{'role':'user',
                 'parts':["sagar paid 20 euro to toran"]}]

messages.append({'role':'model',
                 'parts':["from:toran, money:20 euro, to:sagar"]})
messages.append({'role': 'user',
               'parts': ["toran have give me (yarrko) 40 euro."]})

messages = '''
exampl 1: sagar paid 20 euro to toran
response: "from:toran, money:20 euro, to:sagar"
################
toran have give me (yarrko) 40 euro.
'''



get_chat_response(messages)
               

candidates {
  content {
    role: "model"
    parts {
      text: "\"from:toran, money:40 euro, to:yarrko\""
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.139045298
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.1603975
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.301876307
    severity: HARM_SEVERITY_LOW
    severity_score: 0.306527048
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.196055457
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.14903529
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.199152201
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.137880355
  }
}
usage_metadata {
  prompt_token_count: 55
  candidates_token_count:

# Splitwise

In [6]:
def read_json(file_path):
    """
    Read JSON data from a file.

    Parameters:
        file_path (str): The path to the JSON file.

    Returns:
        dict: The JSON data as a dictionary.
    """
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def write_json(data, file_path):
    """
    Write JSON data to a file.

    Parameters:
        data (dict): The JSON data to write.
        file_path (str): The path to write the JSON file.
    """
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

In [7]:
def get_time():
    # Getting the current date and time
    dt = datetime.now()
    ts = datetime.timestamp(dt)
    return ts

In [21]:
def calculate_net_amount(expense_name='None'):
    data              =  read_json('/home/spanwar/Documents/collage/projects/chatbot_v2/data/splitwise/data.json')
    expense_name_dict = read_json('/home/spanwar/Documents/collage/projects/chatbot_v2/data/splitwise/expense_id.json')

    # try:
    if expense_name=='None':
        expense_name = max(expense_name_dict, key=expense_name_dict.get)
        key          = expense_name_dict[expense_name]   
        print(f'expense name not specified, so calculating last expenses\nexpense name: {expense_name}')
    elif expense_name=='last':
        expense_name = max(expense_name_dict, key=expense_name_dict.get)
        key          = expense_name_dict[expense_name]   
        print(f'last expense name: {expense_name}')
    else:
        key = expense_name_dict[expense_name]
        print(f'expense name: {expense_name}')
    # except:
    #     print('not found')
    #     return

    transactions = data[str(key)]
    net_amounts = {}
    
    # Iterate through each transaction
    for transaction in transactions:
        giver, receiver, amount = transaction
        amount = int(amount)
        
        # Update net amount for the giver and receiver
        net_amounts[giver] = net_amounts.get(giver, 0) - amount
        net_amounts[receiver] = net_amounts.get(receiver, 0) + amount
    
    print(f'Net Balance of the expenses\n{net_amounts}')

    return f'Net Balance of the expenses\n{str(net_amounts)}'

In [16]:
def splitwise(_from, _to, money, expense_name='None'):
    try:
        data              =  read_json('/home/spanwar/Documents/collage/projects/chatbot_v2/data/splitwise/data.json')
        expense_name_dict = read_json('/home/spanwar/Documents/collage/projects/chatbot_v2/data/splitwise/expense_id.json')
    except:
        data = {}
        expense_name_dict = {}
        
    if expense_name=='None':
        print('creating new expense')
        key                         = get_time()
        expense_name_dict[str(key)] = key
        data[key]                   = [[_from, _to, money]]
    elif expense_name=='last':
        print(f'logging to expense name: {expense_name}')
        try:
            expense_name = max(expense_name_dict, key=expense_name_dict.get)
            key          = expense_name_dict[expense_name]
            data[str(key)].append([_from, _to, money])
        except:
            return 'no last expense'
    else:
        print(f'logging to expense name: {expense_name}')
        
        if expense_name in expense_name_dict:
            key = expense_name_dict[expense_name]
            data[str(key)].append([_from, _to, money])
        else:
            key                             = get_time()
            expense_name_dict[expense_name] = key
            data[str(key)]                       = [[_from, _to, money]]

    # print(expense_name_dict)
    # print(data)
    write_json(expense_name_dict, '/home/spanwar/Documents/collage/projects/chatbot_v2/data/splitwise/expense_id.json')
    write_json(data, '/home/spanwar/Documents/collage/projects/chatbot_v2/data/splitwise/data.json')
    print(f"Done: add {money} to {_to} paid by {_from}")

In [211]:
splitwise('A', 'B', '1', expense_name='test')

logging to expense name: test
Done: add 1 euro to B paid by A


In [17]:
calculate_net_amount()

expense name not specified, so calculating last expenses
expense name: 2023 food
Net Balance of the expenses
{'sagar': -3, 'yarkoo': 3}


In [18]:
message = '''

Here's how to use the functions:

splitwise(_from, _to, money, expense_name=None) 
Use this function to record new transactions. Provide the payer (_from), recipient (_to), and the amount of money exchanged. 
Optionally, you can specify an expense_name to categorize the transaction if expense_name is last, it will add to expense name.

calculate_net_amount(expense_name=None): Use this function to calculate the net amounts owed or owed by each individual. 
If an expense_name is provided, the function will calculate net amounts based on transactions associated with that expense name. 
If no expense_name is specified, it will default to the last recorded expense.

example 1: I(sagar) have paid rena 20 euro. add this to last expense.
ans: {"splitwise":{ "from": "sagar", "to":"rena", "money":"20", "expense_name": "last"}}
example 2: ekta give me (liu) 5 euro.
ans: {"splitwise":{"from": "ekta", "to":"liu", "money":"5", "expense_name": "None"}}
example 3: jack have taken 2 euro from pio. add this to expense name ettp tt
ans: {"splitwise":{"from": "pio", "to":"jack", "money":"2", "expense_name": "ettp tt"}}
example: calculate the money that have to paid to each other.
ans: {"calculate_net_amount":{"expense_name": "None"}}
all output in dictionary form.
####################

'''

In [19]:
def call_gemini(text, message):
    message = message + text
    response = get_chat_response(message)
    txt_output = response_to_text(response)
    _dict = json.loads(txt_output.replace('\\"','\"'))
    print(_dict)
    for key in _dict:
        if key=='splitwise':
            splitwise(_dict[key]['from'], _dict[key]['to'], _dict[key]['money'], _dict[key]['expense_name'])
        if key=='calculate_net_amount':
            calculate_net_amount(_dict[key]['expense_name'])
    # return txt_output

In [215]:
call_gemini(
    'liu have give sagar 1 euro last year in summer. add this to expense name year 2023 food.', 
    message)

{'splitwise': {'from': 'liu', 'to': 'sagar', 'money': '1', 'expense_name': 'year 2023 food'}}
logging to expense name: year 2023 food
Done: add 1 to sagar paid by liu


In [225]:
call_gemini(
    'sagar have paid yarkoo 3 euro. Add this to the expense name year 2023 food. give me the net balance of this expense.', 
    message)

{'splitwise': {'from': 'sagar', 'to': 'yarkoo', 'money': '3', 'expense_name': 'year 2023 food'}, 'calculate_net_amount': {'expense_name': 'year 2023 food'}}
logging to expense name: year 2023 food
Done: add 3 to yarkoo paid by sagar
expense name: year 2023 food
Net Balance of the expenses
{'liu': -3, 'sagar': -12, 'yarkoo': 15}


In [22]:
call_gemini(
    'Give me the net balance of my lsat expense.', 
    message)

{'calculate_net_amount': {'expense_name': 'last'}}
last expense name: 2023 food
Net Balance of the expenses
{'sagar': -3, 'yarkoo': 3}


In [22]:
calculate_net_amount(expense_name='None')

expense name not specified, so calculating last expenses
expense name: 2023 food
Net Balance of the expenses
{'sagar': -3, 'yarkoo': 3}


"Net Balance of the expenses\n{'sagar': -3, 'yarkoo': 3}"